In [2]:
%reset -f

import jax
import jax.numpy as jnp

from updec.utils import polyharmonic, multiquadric
from updec.operators import *

In [3]:
RBF = polyharmonic      ## Can define which rbf to use
MAX_DEGREE = 2

cloud = Cloud()
cloud.print_cloud_properties()


=== Meshfree cloud for RBF method ===


Cloud bounding box: Nx = 7  -  Ny = 5

Boundary types (0=internal, 1=dirichlet, 2=neumann):
 {15: 1, 16: 1, 17: 1, 18: 1, 19: 1, 20: 1, 0: 0, 1: 0, 2: 0, 21: 1, 22: 1, 3: 0, 4: 0, 5: 0, 23: 1, 24: 1, 6: 0, 7: 0, 8: 0, 25: 1, 26: 1, 9: 0, 10: 0, 11: 0, 27: 1, 28: 1, 12: 0, 13: 0, 14: 0, 29: 1, 30: 1, 32: 2, 33: 2, 34: 2, 31: 1}
Number of: 
	-Internal points: M = 15 
	-Dirichlet points: MD = 17 
	-Neumann points: MN = 3
Surfaces:
 {15: 2, 16: 2, 17: 2, 18: 2, 19: 2, 20: 1, 0: 0, 1: 0, 2: 0, 21: 3, 22: 1, 3: 0, 4: 0, 5: 0, 23: 3, 24: 1, 6: 0, 7: 0, 8: 0, 25: 3, 26: 1, 9: 0, 10: 0, 11: 0, 27: 3, 28: 1, 12: 0, 13: 0, 14: 0, 29: 3, 30: 1, 32: 4, 33: 4, 34: 4, 31: 3}

Global indices:
 [[15 16 17 18 19]
 [20  0  1  2 21]
 [22  3  4  5 23]
 [24  6  7  8 25]
 [26  9 10 11 27]
 [28 12 13 14 29]
 [30 32 33 34 31]]

Outward normals on Neumann boundaries:
	 32 : [1. 0.]
	 33 : [1. 0.]
	 34 : [1. 0.]

Node coordinates:
	 15 : [0. 0.]
	 16 : [0.   0.25]
	 17 : 

In [3]:
## Operates on radial basis functions and polynomials at position x
def my_diff_operator(x, node, monomial):
    return  -1 * nodal_laplacian(x, node, monomial, rbf=RBF)

known_field = jnp.ones((cloud.N, 2))
## Operates on entire fields at position x (inside node)
def my_rhs_operator(x):
    return divergence(x, known_field, cloud, rbf=RBF, max_degree=2)

all_surfaces = jnp.unique(cloud.surfaces.values())
boundary_functions = {}
for i in all_surfaces:
    if cloud.surface_types[i] == "dirichlet":
        boundary_functions[i] = lambda node: 1.
    elif cloud.surface_types[i] == "neumann":
        boundary_functions[i] = lambda node: 0.

TypeError: unique requires ndarray or scalar arguments, got <class 'dict_values'> at position 0.

In [ ]:
unknown_field = pde_solver(my_diff_operator, my_rhs_operator, cloud, boundary_functions, RBF, MAX_DEGREE)
unknown_field